In [ ]:
from exif import Image
from pathlib import Path

img = Path("/home/pierre/Pictures/Phone-Camera/PXL_20230924_015614422.jpg")
d = Image(open(img, "rb"))

print(d.get("flash"))

def debug(k):
    print(k)
    return k
{
    k: str(d.get(k))
    for k in dir(d)
    if not k.startswith("_") and not callable(d.get(k)) and d.get(k) is not None
}

In [ ]:
from operator import add, sub, mul

s = "- * 3 + 1 2 4"

def bob(l):
    s = l.pop(0)
    if s.isnumeric():
        return int(s)
    return {"+": add, "-": sub, "*": mul}[s](bob(l), bob(l))

bob(s.split())


In [ ]:
from operator import add, sub, mul

s = "- * 3 + 1 2 4"

def bob(l):
    return int(s) if (s:=l.pop(0)).isnumeric() else {"+": add, "-": sub, "*": mul}[s](bob(l), bob(l))

bob(s.split())


In [ ]:
import PIL.ExifTags
import PIL.Image

d = dict(zip(PIL.ExifTags.TAGS.values(), PIL.ExifTags.TAGS.keys()))
d["UserComment"]

img = PIL.Image.open("/home/pierre/repositories/exif-tagger/res/PXL_20231217_005734707.jpg")

In [ ]:
exif = img.getexif()
{
    PIL.ExifTags.TAGS[k]: v
    for k,v in exif.items()
    if k in PIL.ExifTags.TAGS
}

In [ ]:
ifd = exif.get_ifd(PIL.ExifTags.IFD.GPSInfo)
{
    PIL.ExifTags.GPSTAGS[k]: v
    for k,v in ifd.items()
    if k in PIL.ExifTags.GPSTAGS
}

In [ ]:
import pandas as pd
import json

df = pd.DataFrame(
    {"a": list(range(5)), "b": list(range(5,10))}
)

for i,row in df.iterrows():
    print(json.dumps(dict(row)))
    # print(dict(row))


In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    surname: str

Person(*["pierre","louvart"])

In [ ]:
from exif_tagger.database import FaceDatabase,Person
from exif_tagger.img_clipper import infer

In [ ]:
pierre = Person(name="Pierre", surname="Louvart")
henri = Person(name="Henri", surname="Louvart")

db = FaceDatabase(database_path="../db/face-database.sqlite")

face = db.getFaces()[1]
infer(db, face)


In [1]:
from exif_tagger.embeddings import generate_embeddings
embs,names = generate_embeddings("/home/pierre/Downloads/VGG-Face2/data/vggface2_test/small_test")

Running on device: cpu


100%|██████████| 6/6 [00:50<00:00,  8.43s/it]


In [ ]:
import numpy as np
np.save("../res/small_test_embeddings_vg2.npy", embs.numpy())

In [ ]:
import json
json.dump(names, open("../res/names.json", "r"))

In [ ]:
import numpy as np
import json

names = np.array(json.load(open("../res/names.json")))

embs = np.load("../res/embeddings_vg2.npy")

In [ ]:
embs_9 = embs[np.array(names) == "n000009"] # Asian
embs_29 = embs[np.array(names) == "n000029"] # Old White
embs_40 = embs[np.array(names) == "n000040"] # Old White
embs_480 = embs[np.array(names) == "n000480"] # White Woman
embs_2351 = embs[np.array(names) == "n002351"] # Black
embs_928 = embs[np.array(names) == "n000928"] # Black
embs_928.shape

In [ ]:
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import tqdm

In [ ]:
scores_diag = []
scores_full = []
scores_tied = []
scores_spherical = []
X = embs[np.array(names) == "n000480"]
print(X.shape)
# X = embs[np.array(names) == "n000009"]
for i in range(1, 20):
    gm = GaussianMixture(n_components=i, covariance_type="diag")
    gm.fit(X)
    score = gm.score(X)
    scores_diag.append(score)
    gm = GaussianMixture(n_components=i, covariance_type="spherical")
    gm.fit(X)
    score = gm.score(X)
    scores_spherical.append(score)
    gm = GaussianMixture(n_components=i, covariance_type="tied")
    gm.fit(X)
    score = gm.score(X)
    scores_tied.append(score)
    gm = GaussianMixture(n_components=i, covariance_type="full")
    gm.fit(X)
    score = gm.score(X)
    scores_full.append(score)

plt.plot(scores_diag)
plt.plot(scores_full, color="r")
plt.plot(scores_spherical, color="g")
plt.plot(scores_tied, color="orange")


In [ ]:
from sklearn.decomposition import PCA

n_components = 0
means = []
covariances = []
weights = []
precisions_cholesky_ = []

small_embs = PCA(n_components=128).fit_transform(embs)

for name in tqdm.tqdm(set(names)):
    gm = GaussianMixture(n_components=1)
    gm.fit(small_embs[names == name][:])
    n_components += len(gm.means_)
    means += [gm.means_]
    covariances += [gm.covariances_]
    weights += [gm.weights_]
    precisions_cholesky_ += [gm.precisions_cholesky_]
    




In [ ]:
small_embs.shape

In [ ]:
names_to_id_map = {name: i for i,name in enumerate(set(names))}
classes = [names_to_id_map[name] for name in names]


In [ ]:
from joblib import dump

model = GaussianMixture(n_components=n_components)
model.means_ = np.concatenate(means, axis=0)
model.covariances_ = np.concatenate(covariances, axis=0)
model.precisions_cholesky_ = np.concatenate(precisions_cholesky_, axis=0)
model.weights_ = np.ones((n_components,)) / np.sum(n_components)
dump(model, "../res/knn_model.joblib", compress=9)
np.mean(model.predict(small_embs[len(names)//2:]) == classes[len(names)//2:])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC

SVC = GaussianProcessClassifier()
# model = GaussianProcessClassifier()
# model = KNeighborsClassifier(n_neighbors=5)

mean_embs = []
model.fit(embs[::50], classes[::50])

In [ ]:
np.mean(model.predict(embs[2::50]) == classes[2::50])

In [ ]:
from joblib import dump

dump(model, "../res/knn_model.joblib")

In [ ]:

beg = "({["
end = ")}]"
valid_pairs = list(zip(beg, end))

def valid_rec(l: list[str]) -> tuple[str, bool]:
    x,s = "",True
    while len(l) and (x:=l.pop(0)) in beg:
        y,ss = valid_rec(l)
        x,s = "",s and ss and (x, y) in valid_pairs
    return (x, s)
    
def valid(t: str):
    return valid_rec(list(t)) == ("", True)


t1 = "[]" # valid
t2 = "[({([{}])})]]" # invalid
t3 = "[({([{}])})][()[{()[]{}}]]" # valid
t4 = "[({([{}])})][()[{()[]{}}]])" # invalid
t5 = "{[({([{}])})][()[{()[]{}}]]" # invalid

valid(t1), valid(t2), valid(t3), valid(t4), valid(t5),

In [ ]:
f = "acdjcb   ask\n  kkd\t\n"
f.strip()